# Simple example
This notebook provides a brief overview of PyJobShop's modeling features for solving scheduling problems, specifically:

- How to model a simple scheduling problem using PyJobShop's `Model` interface,
- How to inspect the resulting solution, and
- How to plot the found solution.

In [ ]:
from pyjobshop import Model

model = Model()

Let's add some data to the model. We add four jobs, each consisting of two tasks.

In [ ]:
jobs = [model.add_job() for _ in range(4)]
tasks = [[model.add_task(job=job) for _ in range(2)] for job in jobs]

Passing the `job` argument to `m.add_task(job=job)` ensures that the created task belongs to the right job.
Next, we have to create the machines and also specify processing times for each task. The duration of an task is equal to the job index plus one (1, 2, 3 or 4).

In [ ]:
machines = [model.add_machine() for _ in range(2)]

for job in range(len(jobs)):
    for task in tasks[job]:
        for machine in machines:
            duration = job + 1
            model.add_processing_time(machine, task, duration)

Now that we have defined all important elements, we can now solve this model. By default, the model aims to minimize the makespan, which is the maximum completion time of all jobs.

In [ ]:
result = model.solve(log=False)
result.status

We found the optimal solution!

## Solution

The result variables stores a `Result` object that contains information about the solving progress and the best found solution. A solution consists of `Task`s, which stores for each task the assigned machine, the start time, and the duration.

In [ ]:
for task in result.best.schedule:
    print(task)

## Plotting

Each scheduled task can now be nicely plotted in a Gantt chart.

In [ ]:
from pyjobshop import plot

data = model.data()
plot(data, result.best)

The plot shows a Gantt chart of our solution. Each row represents a machine and each horizontal bar represents a scheduled task.
The colors of the tasks depict the individual jobs they are associated with, with each job having a unique color.

## Conclusion

This concludes this example. We showed how to use PyJobShop's Model interface to setup a simple example, and we showed the representation of a solution and how it can be plotted. For more details about `Model`, see the [API documentation](https://pyjobshop.readthedocs.io/en/latest/api/pyjobshop.html#pyjobshop.Model.Model). In the next notebooks, we show you how to model classical scheduling models.